In [99]:
import json
import numpy as np
import pandas as pd
from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from pymorphy2 import MorphAnalyzer
# лемматизация и мешок слов
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
#удаляю русские стоп слова (расширить???)
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
english_stopwords = stopwords.words("english")
import spacy
from spacy.lang.ru import Russian
from datetime import date
import datetime

In [8]:
with open("case_2_data_for_members.json", "r", encoding="utf-8") as f:    
    data = json.load(f)

In [10]:
data[0].keys()

dict_keys(['vacancy', 'failed_resumes', 'confirmed_resumes'])

In [13]:
data[0]['vacancy']['description']

' Описание Мы расширяем команды и ищем разработчиков для развития нескольких сервисов:   Инвестиции. Мы — лидер среди брокеров по количеству активных клиентов. Делаем инвестиции удобными, технологичными и понятными;   Бизнес. Меняем подход к ведению бухгалтерии, обмену документами между организациями и работе с архивами — переводим все и вся в цифру;   Страхование. Развиваем платформу прямых продаж. Наша цель — предсказуемый, удобный процесс для бизнеса и клиента; Платежные технологии и процессинг. Занимаемся разработкой и поддержкой платежного шлюза банка. Задачи шлюза — определять тип платежей, наполнять их данными из других систем банка и проверять разрешенность операций. У нас много интересных и разнообразных задач, опытная команда и отличные возможности для роста. Откликайтесь на вакансию, чтобы узнать о проектах и выбрать подходящий для вас. Требования Опыт разработки на Java от 3 лет Опыт коммерческой разработки на Java 11+ или Kotlin Опыт коммерческой разработки с любым из фрей

In [22]:
data[2]['vacancy']

{'uuid': 'c03085c3-9b1e-3564-bb1e-59aa72e5fbca',
 'name': 'Ведущий/ Главный аналитик DWH',
 'keywords': None,
 'description': ' Желательно знания Oracle   сегодня — не просто банк. Это сервисы, которыми пользуются люди с разными потребностями, интересами и желаниями. И для каждого нашего клиента мы стараемся сделать удобные и качественные ИТ-продукты. Без сильной, экспертной ИТ-команды это было бы невозможно, поэтому у нас всегда есть интересные проекты для проверки твоих скиллов, собственная IT-академия для начинающих и классный мерч, который можно получить за хорошую работу. Сейчас у нас открыта вакансия Ведущий аналитик DWH в департамент аналитики. Чем предстоит заниматься: – Участие в обсуждении инициатив бизнес-пользователей с точки зрения их возможной реализации в информационных системах; – Интервьюирование пользователей, сбор бизнес-требований; – Исследование информационных систем Банка и интервьюирование технических владельцев для определения источников данных, их свойств и лог

In [17]:
data[0]['confirmed_resumes'][0]['experienceItem'][0]['description']

' Проекты: Внутренний продукт  а по заказу инструментов разработки для сотрудников; Команда состояла из владельца продукта, аналитика, 4 бэкенд разработчиков, 4 фронтенд разработчиков, 1 тестер. Что делал на проектах: Спроектировал и реализовал новые методы REST API, разрабатывал новый бизнес функционал, исправлял баги; Создавал новые микро-сервисы на SpringBoot, покрыл их тестами и передал пользователям; Покрыл проект юнит-тестами с использованием jUnit, Mockito; Работал в команде, проходил и проводил code-review. Технологии: Java 11, Spring Boot, Hibernate, Spring Data, Spring Security, Maven, Camunda, Postgres, Liquibase, Kafka, jUnit, Mockito, Swagger, Bitbucket '

In [19]:
data[0]['confirmed_resumes'][0]

{'uuid': '8c8cf797-2c6b-3f4b-b28b-20d57bd88b82',
 'first_name': 'Марк',
 'last_name': 'Силин',
 'birth_date': '1987-01-01',
 'country': 'Россия',
 'city': 'Великий Новгород',
 'about': None,
 'key_skills': 'Java, Java Spring Framework, Hibernate, Git, PostgreSQL, REST, Camunda, Apache Kafka, Spring Boot',
 'experienceItem': [{'starts': '2022-02-01',
   'ends': None,
   'employer': 'ОАО ЭлектроАвтоТяж',
   'city': 'Коломна',
   'position': 'java-разработчик',
   'description': ' Проекты: Внутренний продукт  а по заказу инструментов разработки для сотрудников; Команда состояла из владельца продукта, аналитика, 4 бэкенд разработчиков, 4 фронтенд разработчиков, 1 тестер. Что делал на проектах: Спроектировал и реализовал новые методы REST API, разрабатывал новый бизнес функционал, исправлял баги; Создавал новые микро-сервисы на SpringBoot, покрыл их тестами и передал пользователям; Покрыл проект юнит-тестами с использованием jUnit, Mockito; Работал в команде, проходил и проводил code-review

In [25]:
len(data[10]['failed_resumes'])

13

In [39]:
import json
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#nltk.download('punkt')
#nltk.download('wordnet')

# Загрузка данных
resume = data[4]['confirmed_resumes'][0]['experienceItem'][0]['description']
#resume = data[2]['failed_resumes'][0]['experienceItem'][0]['description']
vacancy = data[4]['vacancy']['description']

# Предобработка nltk.corpus.stopwords.words("russian")
#stop_words = set(["the", "is", "of", "and", "a", "to", "in", "on", "with"])
stop_words = nltk.corpus.stopwords.words("russian")
lemmatizer = WordNetLemmatizer()

resume_text = " ".join([lemmatizer.lemmatize(token) for token in word_tokenize(resume.lower()) if token not in stop_words])
vacancy_text = " ".join([lemmatizer.lemmatize(token) for token in word_tokenize(vacancy.lower()) if token not in stop_words])

# Сопоставление
tfidf_vectorizer = TfidfVectorizer()
vectors = tfidf_vectorizer.fit_transform([resume_text, vacancy_text])
cosine_similarity_score = cosine_similarity(vectors[0], vectors[1])

# Вывод
print("Косинусное сходство:", cosine_similarity_score)

Косинусное сходство: [[0.01059969]]


In [58]:
def get_cosinus(resume, vacancy):
    stop_words = nltk.corpus.stopwords.words("russian")
    lemmatizer = WordNetLemmatizer()

    resume_text = " ".join([lemmatizer.lemmatize(token) for token in word_tokenize(resume.lower()) if token not in stop_words])
    vacancy_text = " ".join([lemmatizer.lemmatize(token) for token in word_tokenize(vacancy.lower()) if token not in stop_words])

    # Сопоставление
    tfidf_vectorizer = TfidfVectorizer()
    vectors = tfidf_vectorizer.fit_transform([resume_text, vacancy_text])
    cosine_similarity_score = cosine_similarity(vectors[0], vectors[1])

    return cosine_similarity_score[0][0]

In [93]:
def calculate_age(birthdate):
    today = date.today()
    year_diff = today.year - birthdate.year
    month_diff = today.month - birthdate.month
    day_diff = today.day - birthdate.day

    if month_diff < 0:
        year_diff -= 1
    elif month_diff == 0 and day_diff < 0:
        year_diff -= 1

    return year_diff

In [135]:
def get_distance(resume):
    res = 0
    try:
        country = resume['country']
        city = resume['city']
        if(country=='Россия'):
            res += 2
        if(city=='Санкт-Петербург'):
            res += 1
        elif (city=='Москва'):
            res += 2
    except:
        res = 0
    return res       
    

In [150]:
def get_experience_count(resume):
    res = 0
    try:
        res = len(resume['experienceItem'])
    except:
        res = 0
    return res

In [163]:
def get_experience_age(resume):
    res = 0
    try:
        for exp in resume['experienceItem']:
            starts = pd.to_datetime(exp['starts'])
            ends = pd.to_datetime(exp['ends'])
            if ends == None:
                ends = date.today()
            year_diff = ends.year - starts.year
            month_diff = ends.month - starts.month
            day_diff = ends.day - starts.day

            if month_diff < 0:
                year_diff -= 1
            elif month_diff == 0 and day_diff < 0:
                year_diff -= 1
                
            res += year_diff    
    except:
        res = 0
    return res
            

In [180]:
def get_resume_length(resume):
    # Складываем все описания предыдущих мест работы и получаем длину
    res = 0
    str = ''
    try:
        for exp in resume['experienceItem']:
            str += exp['description']
    except:
        None
    res = len(str)
    return res

In [193]:
def get_skills_length(resume):
    res = 0
    try:
        res = len(resume['key_skills'])
    except:
        None
    return res

In [234]:
# Создаем датафрейм для модели
#df = pd.DataFrame(columns=["uuid", "age", "distance", "experience_count", "experience_age", "resume_length", "cosinus", "success"])
df = pd.DataFrame(columns=["uuid", "age", "distance", "experience_count", "experience_age", "resume_length", "skills", "cosinus", "success"])
#df = pd.DataFrame(columns=["uuid", "age", "distance", "resume_length", "skills", "cosinus", "success"])

In [43]:
data[0]["failed_resumes"][0]

{'uuid': '74392e00-ecfb-335b-9fc1-c2652dca06e5',
 'first_name': 'Данила',
 'last_name': 'Прохоров',
 'birth_date': '1979-01-01',
 'country': 'Россия',
 'city': 'Санкт-Петербург',
 'about': None,
 'key_skills': 'Java, Spring Boot, Java EE, SQL, Hibernate, Git, Docker, Высоконагруженные системы, PostgreSQL, Linux',
 'experienceItem': [{'starts': '2022-05-01',
   'ends': '2023-10-23',
   'employer': 'МФО ХмельИнфоОрион',
   'city': 'Можайск',
   'position': 'Ведущий инженер-программист',
   'description': ' Интеграционные адаптеры для передачи заявок пользователей на API компаний партнеров (Кредиты, ипотека, кредитные карты) и возврат полученных ответов системам Банки.ру. Участие в обсуждении архитектурных решений. Проведение code-review. Обучение сотрудников (наставничество). Стек: Java, Docker, SQL, Hibernate, Java Spring Framework, Apache Maven, REST, Redis, Системная интеграция '},
  {'starts': '2021-08-01',
   'ends': '2022-05-01',
   'employer': 'МКК РадиоITАсбоцементМаш',
   'city'

In [237]:
def get_features(resume):
    features = []
    features.append(resume['uuid'])  # uuid
    age = 20
    try:
        birth_date = pd.to_datetime(resume['birth_date'])
        age = calculate_age(birth_date)
    except:
        age=20    
    features.append(age) # Возраст
    
    distance = get_distance(resume)
    features.append(distance)  # Премия за географическое положение
    
    experience_count = get_experience_count(resume)
    features.append(experience_count)  #  Количество мест предыдущих работ
    
    experience_age = get_experience_age(resume)
    features.append(experience_age)  #  Стаж
    
    resume_length = get_resume_length(resume)
    features.append(resume_length)  #  Длина резюме
    
    skills = get_skills_length(resume)
    features.append(skills)  #  Длина скиллов в описании
    
    try:
        experience = resume['experienceItem'][0]['description']
    except:
        experience = None
    if experience:
        cosinus = get_cosinus(experience, vacancy_description)
    else:
        cosinus = 0
    features.append(cosinus)  # Косинусное расстояние от описания последнего опыта работы и вакансии
    
    return features
        

In [238]:
# Идем по всем вакансиям и по всем пользователям
for index, vacancy in enumerate(data):
    vacancy_description = vacancy['vacancy']['description']  # Описание вакансии
    # Идем по всем отвергнутым кандидатам
    for resume in vacancy["failed_resumes"]:
        # features        
        features = get_features(resume)        
        features.append(0)  # неуспешный кандидат
        df.loc[len(df)] = features
        
    for resume in vacancy["confirmed_resumes"]:
        # features        
        features = get_features(resume)
        features.append(1)  # успешный кандидат
        df.loc[len(df)] = features
    

In [239]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 656 entries, 0 to 655
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   uuid              656 non-null    object 
 1   age               656 non-null    int64  
 2   distance          656 non-null    int64  
 3   experience_count  656 non-null    int64  
 4   experience_age    656 non-null    int64  
 5   resume_length     656 non-null    int64  
 6   skills            656 non-null    int64  
 7   cosinus           656 non-null    float64
 8   success           656 non-null    int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 51.2+ KB


In [241]:
df.tail()

,uuid,age,distance,experience_count,experience_age,resume_length,skills,cosinus,success
651,2ddef29c-dd38-33d9-b046-7099a548c570,29,2,3,3,1616,134,0.079917,1
652,88779779-a59e-3580-b451-77f40ecd6c0e,35,2,4,6,1074,77,0.072100,1
653,3c418fd4-8956-31e3-a5ef-f638f6eac0c9,31,2,4,3,1752,241,0.104498,1
654,253cac7a-6788-359a-b10a-c444dcfc049e,24,3,4,5,867,126,0.005093,1
655,76e284ec-e1a0-3909-a063-b9f8ff0ad242,26,4,5,5,2778,168,0.075998,1


In [242]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [287]:
params = {
        'verbose': 100,
        'random_seed': 42,
        'learning_rate': 0.1,
        'eval_metric': 'F1',        
        'iterations':2000
        #'cat_features': cat_features, 
}

In [288]:
X = df.drop(["uuid","success"], axis=1)
y = df["success"]

In [289]:
model = CatBoostClassifier(**params)

In [290]:
# Разделяем датафрейм на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [291]:
model.fit(X_train, y_train)

0:	learn: 0.1625000	total: 2.45ms	remaining: 4.9s
100:	learn: 0.9132075	total: 248ms	remaining: 4.67s
200:	learn: 0.9929078	total: 458ms	remaining: 4.1s
300:	learn: 0.9928571	total: 655ms	remaining: 3.7s
400:	learn: 0.9928571	total: 853ms	remaining: 3.4s
500:	learn: 0.9929078	total: 1.07s	remaining: 3.21s
600:	learn: 0.9929078	total: 1.27s	remaining: 2.96s
700:	learn: 0.9928571	total: 1.47s	remaining: 2.72s
800:	learn: 0.9929078	total: 1.66s	remaining: 2.49s
900:	learn: 0.9929577	total: 1.86s	remaining: 2.26s
1000:	learn: 0.9929078	total: 2.04s	remaining: 2.04s
1100:	learn: 0.9929078	total: 2.24s	remaining: 1.83s
1200:	learn: 0.9929078	total: 2.43s	remaining: 1.62s
1300:	learn: 0.9928571	total: 2.62s	remaining: 1.41s
1400:	learn: 0.9928571	total: 2.81s	remaining: 1.2s
1500:	learn: 0.9928571	total: 3s	remaining: 996ms
1600:	learn: 0.9929078	total: 3.19s	remaining: 795ms
1700:	learn: 0.9929078	total: 3.38s	remaining: 595ms
1800:	learn: 0.9929577	total: 3.58s	remaining: 396ms
1900:	learn:

In [292]:
score = model.score(X_test, y_test)
score

0.700507614213198

In [296]:
model.save_model("model.cbm")

In [297]:
prediction = model.predict(X)

In [298]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y, prediction)

print("Accuracy:", accuracy)

Accuracy: 0.9070121951219512


In [299]:
from sklearn.metrics import precision_score

precision = precision_score(y, prediction)

print("Precision:", precision)

Precision: 0.8707865168539326


In [300]:
from sklearn.metrics import f1_score

f1 = f1_score(y, prediction)

print("F1:", f1)

F1: 0.8355795148247979


In [293]:
# Загружаем файл с вакансией и кандидатами
with open("case_2_reference_without_resume_sorted.json", "r", encoding="utf-8") as f:    
    data2 = json.load(f)

In [ ]:
data2

In [318]:
# Создаем датафрейм для соискателей
df2 = pd.DataFrame(columns=["uuid", "age", "distance", "experience_count", "experience_age", "resume_length", "skills", "cosinus"])


In [319]:
vacancy_descr = data2['vacancy']['description']
print(len(data2['resumes']))
for resume in data2['resumes']:
    # features        
    features = get_features(resume)            
    df2.loc[len(df2)] = features


113


In [320]:
df2.tail()

,uuid,age,distance,experience_count,experience_age,resume_length,skills,cosinus
108,82df355a-235e-3046-9e6e-782ddf1600eb,28,4,2,2,2572,0,0.132429
109,915597ce-24e5-31fa-8dca-29437f49f839,37,2,9,8,2278,346,0.033970
110,f288a532-0b58-30cb-ac3c-f87e53984719,39,4,3,5,1427,79,0.035326
111,3e3a379f-226e-305e-b7d8-cf341e00cbd7,30,0,4,2,1168,225,0.031713
112,cc88bf96-f0b9-313a-abce-dbe60b6f1c98,36,4,3,14,1695,29,0.000000


In [321]:
X2 = df2.drop("uuid", axis=1)
X2

,age,distance,experience_count,experience_age,resume_length,skills,cosinus
0,32,4,5,3,1712,77,0.045842
1,33,2,4,10,4629,419,0.023021
2,34,3,3,3,923,129,0.039635
3,34,4,6,7,1538,200,0.039982
4,29,4,6,7,3163,0,0.004587
...,...,...,...,...,...,...,...
108,28,4,2,2,2572,0,0.132429
109,37,2,9,8,2278,346,0.033970
110,39,4,3,5,1427,79,0.035326
111,30,0,4,2,1168,225,0.031713


In [322]:
prediction = model.predict(X2)

In [326]:
candidates = df2[prediction==1]

In [328]:
candidates

,uuid,age,distance,experience_count,experience_age,resume_length,skills,cosinus
3,9a9c3ff1-49f8-30dd-a294-e56fc60cae64,34,4,6,7,1538,200,0.039982
4,6561771c-7ef3-3e50-ab3a-ba8547201480,29,4,6,7,3163,0,0.004587
7,8e456229-6e4e-3d57-a90e-ae4c3e1db869,29,4,3,5,910,82,0.020484
9,5785c202-6744-3e1b-994a-d5bffc6aad14,24,0,3,5,2761,119,0.034054
17,aff4b709-ccd2-340a-949f-84b1bb7eb260,28,0,3,4,3202,93,0.069015
18,9cef8747-af3a-3833-9098-3bfabf1e40eb,35,0,7,5,1903,215,0.000000
19,c2a6bf5a-45ca-3f79-93ff-355a724b65df,31,4,6,3,2045,169,0.006746
21,1f1809fd-fde5-3720-a5dc-02f1160f53a9,28,3,1,2,381,66,0.045735
25,42eae4bf-4826-3105-b197-dc0afb063714,32,2,4,3,2100,90,0.076879
28,0cfa3d00-10b5-38d5-b166-329ff6f4598b,39,0,2,3,1295,284,0.064620


In [329]:
candidates["uuid"].to_csv("uuid_123456.csv", index=False)